In [61]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam
import pandas as pd
import numpy as np
from predict import date_to_int, train_test_split, create_sequences, advisable
from sklearn.preprocessing import MinMaxScaler

TRAIN_RATIO = 0.8
TIMESTEP = 60
FORECAST_STEPS = 10

In [48]:
df = pd.read_csv('data/yf_daily/AAPL.csv')
df = df[['Close']]
X_sc = MinMaxScaler()
scaled_X = X_sc.fit_transform(df)

y_sc = MinMaxScaler()
scaled_y = y_sc.fit_transform(df[['Close']])

In [49]:
X, y = create_sequences(scaled_X, scaled_y, TIMESTEP)
X_train, y_train, X_test, y_test = train_test_split(X, y, TRAIN_RATIO)

In [50]:
def create_model(input_shape):
    
    model = Sequential()
    model.add(LSTM(units=50, input_shape=input_shape, return_sequences=False))
    model.add(Dense(units=1))
    model.compile(optimizer=Adam(learning_rate=0.005),loss='mean_squared_error')
        
    return model

In [51]:
model = create_model((X_train.shape[1], X_train.shape[2]))
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
271/271 [==============================] - 7s 17ms/step - loss: 3.3870e-05
Epoch 2/10
271/271 [==============================] - 5s 17ms/step - loss: 1.4478e-06
Epoch 3/10
271/271 [==============================] - 4s 16ms/step - loss: 1.4423e-06
Epoch 4/10
271/271 [==============================] - 4s 16ms/step - loss: 1.7286e-06
Epoch 5/10
271/271 [==============================] - 4s 16ms/step - loss: 1.6165e-06
Epoch 6/10
271/271 [==============================] - 5s 17ms/step - loss: 1.3737e-06
Epoch 7/10
271/271 [==============================] - 5s 18ms/step - loss: 1.4595e-06
Epoch 8/10
271/271 [==============================] - 4s 16ms/step - loss: 1.6262e-06
Epoch 9/10
271/271 [==============================] - 4s 16ms/step - loss: 1.1562e-06
Epoch 10/10
271/271 [==============================] - 5s 17ms/step - loss: 1.2115e-06


In [52]:
loss = model.evaluate(X_test, y_test)
print("Test Loss:", loss)

68/68 [==============================] - 1s 8ms/step - loss: 1.2145e-04
Test Loss: 0.00012144879292463884


In [53]:
scaled_test_predictions = model.predict(X_test)
test_predictions = y_sc.inverse_transform(scaled_test_predictions)

test_targets = y_sc.inverse_transform(y_test)

68/68 [==============================] - 1s 7ms/step


In [64]:
last_sequence = np.reshape(X_test[-1], (1, X_test[-1].shape[0], X_test[-1].shape[1]))
prediction = model.predict(last_sequence)
prediction
X_test[-1][1:]

1/1 [==============================] - 0s 29ms/step


array([[0.97510863],
       [0.98283359],
       [0.99924269],
       [1.        ],
       [0.9972736 ],
       [0.98879132],
       [0.99409274],
       [0.98343944],
       [0.98268205],
       [0.97722925],
       [0.97445231],
       [0.97495716],
       [0.97712825],
       [0.97182684],
       [0.93703956],
       [0.93002152],
       [0.91820699],
       [0.9145212 ],
       [0.93663563],
       [0.93451508],
       [0.9398165 ],
       [0.9367871 ],
       [0.93845326],
       [0.92689119],
       [0.92209463],
       [0.95213595],
       [0.96692935],
       [0.97869342],
       [0.98520653],
       [0.98177328],
       [0.98010712],
       [0.97127145],
       [0.96778766],
       [0.94915701],
       [0.93077883],
       [0.94319929],
       [0.93809987],
       [0.94733939],
       [0.95551874],
       [0.95607413],
       [0.95057079],
       [0.95324673],
       [0.94466345],
       [0.93401016],
       [0.92951659],
       [0.92805243],
       [0.92022654],
       [0.916

In [65]:
def forecast(model, X_test, forecast_steps):
    shape1, shape2 = X_test[-1].shape[0], X_test[-1].shape[1]
    
    for i in range(forecast_steps):
        #get prediction for last timesequence
        last_sequence = np.reshape(X_test[-1], (1, shape1, shape2))
        prediction = model.predict(last_sequence)
        
        #create new instance for X_test set
        new_instance = np.append(X_test[-1][1:], prediction)
        
        X_test = np.append(X_test[1:], new_instance)
    
    return prediction
#TODO predict
    

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(16, 9), dpi=80)
plt.plot(test_targets, color = 'blue', label = 'Real Stock Prices')
plt.plot(test_predictions, color = 'green', label = 'Predicted Stock Prices')
plt.title('Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.show()